In [1]:
import optuna
from backend.genetic_algorithm import *
from math import floor

/Users/stefannosek/Documents/GitHub/class_scheduling/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rand.seed(240)

In [7]:
rooms = [
    Room("B9-F1", 60, "normal"),
    Room("B9-F2", 60, "normal"),
    Room("B9-F3", 60, "normal"),
    Room("B9-F4", 60, "normal"),
    Room("B9-F5", 60, "normal"),
    Room("B9-F6", 30, "normal"),
    Room("B9-F7", 60, "normal"),
    Room("B9-F8", 60, "normal"),
    Room("B9-F9", 60, "normal"),
    Room("B9-F10", 150, "normal"),
    Room("B9-51", 30, "laboratories"),
    Room("B9-52", 60, "normal"),
    Room("B9-53", 30, "laboratories"),
    Room("B14-1.11", 60, "normal"),
    Room("B14-1.12", 30, "laboratories"),
    Room("B14-2.1", 30, "laboratories"),
    Room("B14-2.2", 30, "laboratories"),
    Room("B14-2.3", 30, "laboratories"),
]

program = Data("program.xlsx")

professors = program.create_professors()

courses = program.create_courses(professors)

MS_sem_1_gr_1 = Students(0, 1, 1, "matematyka stosowana")
MS_sem_1_gr_2 = Students(1, 1, 2, "matematyka stosowana")
MS_sem_2_gr_1 = Students(2, 2, 1, "matematyka stosowana")
MS_sem_2_gr_2 = Students(3, 2, 2, "matematyka stosowana")
MS_sem_3_gr_1 = Students(4, 3, 1, "matematyka stosowana")
MS_sem_3_gr_2 = Students(5, 3, 2, "matematyka stosowana")
MS_sem_4_gr_1 = Students(6, 4, 1, "matematyka stosowana")
MS_sem_4_gr_2 = Students(7, 4, 2, "matematyka stosowana")
MS_sem_5_gr_1 = Students(8, 5, 1, "matematyka stosowana")
MS_sem_6_gr_1 = Students(9, 6, 1, "matematyka stosowana")

students = [
    MS_sem_1_gr_1, MS_sem_1_gr_2, MS_sem_2_gr_1, MS_sem_2_gr_2,
    MS_sem_3_gr_1, MS_sem_3_gr_2, MS_sem_4_gr_1, MS_sem_4_gr_2,
    MS_sem_5_gr_1, MS_sem_6_gr_1, 
]

In [17]:
def objective(trial):
    population_size = trial.suggest_int("population_size", 50, 500, step = 10)
    mutation_probability = trial.suggest_float("mutation_probability", 0.1, 1, step = 0.05)
    elites_percentage = trial.suggest_float("elites_percentage", 0.05, 0.2, step = 0.05)
    pairs_percentage = trial.suggest_float("pairs_percentage", 0.4, 1, step = 0.1)
    number_of_class_swaps = trial.suggest_int("number_of_class_swaps", 1, 15, step = 1)
    number_of_room_swaps = trial.suggest_int("number_of_room_swaps", 1, 15, step = 1)

    pop = Population(
        population_size,
        professors,
        courses,
        rooms,
        students,
        mutation_probability,
        floor(population_size * elites_percentage),
        floor(population_size/2 * pairs_percentage), 
        number_of_class_swaps,
        number_of_room_swaps,
    )

    pop.population = pop.sort_by_fitness_and_soft_fitness(pop.population)

    pop.execute(100)
    
    return 5 * pop.population[0].fitness + pop.population[0].soft_fitness

In [18]:
study = optuna.create_study(direction = "maximize")

[I 2024-01-24 23:47:48,412] A new study created in memory with name: no-name-2e0b8383-3d68-403d-925c-47f163996758


In [19]:
study.optimize(objective, n_trials = 100)

[I 2024-01-24 23:50:04,162] Trial 0 finished with value: 5.211538461538462 and parameters: {'population_size': 200, 'mutation_probability': 0.30000000000000004, 'elites_percentage': 0.05, 'pairs_percentage': 1.0, 'number_of_class_swaps': 8, 'number_of_room_swaps': 8}. Best is trial 0 with value: 5.211538461538462.
[I 2024-01-24 23:50:50,401] Trial 1 finished with value: 2.635593220338983 and parameters: {'population_size': 100, 'mutation_probability': 0.30000000000000004, 'elites_percentage': 0.05, 'pairs_percentage': 0.6000000000000001, 'number_of_class_swaps': 5, 'number_of_room_swaps': 12}. Best is trial 0 with value: 5.211538461538462.
[I 2024-01-24 23:53:12,879] Trial 2 finished with value: 5.214285714285714 and parameters: {'population_size': 250, 'mutation_probability': 0.25, 'elites_percentage': 0.05, 'pairs_percentage': 0.8, 'number_of_class_swaps': 6, 'number_of_room_swaps': 10}. Best is trial 2 with value: 5.214285714285714.
[I 2024-01-24 23:55:44,469] Trial 3 finished with 

In [20]:
best_trial = study.best_trial

In [21]:
best_params = best_trial.params

In [22]:
best_params

{'population_size': 480,
 'mutation_probability': 0.9500000000000001,
 'elites_percentage': 0.2,
 'pairs_percentage': 0.5,
 'number_of_class_swaps': 5,
 'number_of_room_swaps': 2}